# LOL Silver Rank Data Scraping

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import time

## LOL api 발급
LOL Developer Portal: https://developer.riotgames.com/

In [3]:
api_key = 'RGAPI-de4642d1-97ce-4b39-8790-c3efa5ce60c8' # Riot Developer에서 발급받은 api

'Rv20tA_dsjnTijCYJAeVVTubpDQCIJkKPnpmiKUrbfBwxQ'

### Silver Rank의 User game ID 추출("summonerName")
* https://developer.riotgames.com/apis#league-v4

In [23]:
league_df = pd.DataFrame()
for i in range(1,10):
    silver = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/SILVER/III?page={0}&api_key='.format(i) + api_key
    r = requests.get(silver)#silver 티어 호출
    if r.status_code == 200:
        league_df = league_df.append(pd.DataFrame(r.json()))
    else:
        break
league_df.reset_index(inplace=True, drop = True)
league_df = league_df.drop(["leagueId","leaguePoints","wins","losses","veteran","inactive","freshBlood","hotStreak"], axis = 1)
print(r.status_code)
league_df2 = league_df[:100].copy() # test data 용도로 사용할 데이터이기 때문에 전부 긁어올 필요가 없으므로 100개만 따로 추출
league_df2

200


,queueType,tier,rank,summonerId,summonerName
0,RANKED_SOLO_5x5,SILVER,III,y-GIH5wDrGZt3rUZ-Y-tcJwWnuaoBKDp87GAn73sFQ74QheZ,Vivre sans aime
1,RANKED_SOLO_5x5,SILVER,III,j_OfQvASOhf4iceHqRDnGr3P63m3cvUWEh-Kpc5Jaya4f6O7,sadiness
2,RANKED_SOLO_5x5,SILVER,III,UcZ40RcMjnBhzbRfO--2tnqk1OcYvkJiiRx5xAvdulQx_Dnr,스웨이이
3,RANKED_SOLO_5x5,SILVER,III,flkyRoBSynNfGh-xFarzBAWZMex0umfiKlJcW_gfNYhaLTbx,나는미래의대머리
4,RANKED_SOLO_5x5,SILVER,III,zp2zKTAMW1ubsdh0poGqEtu3fdrjf7pFijHykilKgakBBmv3,졸라못한 내동생
...,...,...,...,...,...
1840,RANKED_SOLO_5x5,SILVER,III,fw2pW4mbNW7HlcmZVNRi99Q0BUydiAij-OAf-dnBuwy2w6Ih,DWG Tuxedo
1841,RANKED_SOLO_5x5,SILVER,III,T42D7hvH52nqY1S41hjT8gTUpMOifIV_AE8vG-P7BqaHVH88,야스오하는루키
1842,RANKED_SOLO_5x5,SILVER,III,ryfibbkUwUVP_3uHnjbHTizLaQw3e7pSJSHOwaAUz5E5DR6W,VVINTER WIND
1843,RANKED_SOLO_5x5,SILVER,III,sNksFLx7TU7nJFbTbSs3plnpYWEdPSGcPXxyU5-UX1TsaFMb,사그부


### "summonerName" to extract "accountId"
* league_df2 에 담겨있는 "summonerName"을 통해 이젠 'accountId' 수집할 수 있다. 'accountId'는 추후 해당 유저가 속했던 경기의 고유키 값을 수집하는데 사용 된다.
* https://developer.riotgames.com/apis#summoner-v4

In [21]:
for i in range(len(league_df2)):
    try:
        user_pk = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df2['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(user_pk)
        print(i, "request_first: ",r.status_code)
        while r.status_code == 429:
            print(r.status_code)
            time.sleep(120)
            user_pk = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df2['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(user_pk)
            
        account_id = r.json()['accountId']
        league_df2.loc[i, "account_id"] = account_id
    
    except:
        pass
silver_3 = league_df2.copy()

0 request_first:  200
1 request_first:  200
2 request_first:  200
3 request_first:  200
4 request_first:  200
5 request_first:  200
6 request_first:  200
7 request_first:  200
8 request_first:  200
9 request_first:  200


In [22]:
silver_3

,queueType,tier,rank,summonerId,summonerName,account_id
0,RANKED_SOLO_5x5,SILVER,III,y-GIH5wDrGZt3rUZ-Y-tcJwWnuaoBKDp87GAn73sFQ74QheZ,Vivre sans aime,T_XlZJBVbX8spefFXnphPTA9ALjnoEz_D3wyUUxHsGwEve...
1,RANKED_SOLO_5x5,SILVER,III,j_OfQvASOhf4iceHqRDnGr3P63m3cvUWEh-Kpc5Jaya4f6O7,sadiness,OBEjsiST9JLEqxfcPQnNpGvryE_AoZIuAHzGdRgraJbZTf...
2,RANKED_SOLO_5x5,SILVER,III,UcZ40RcMjnBhzbRfO--2tnqk1OcYvkJiiRx5xAvdulQx_Dnr,스웨이이,aJP4O8vOGoeiKr1odS4Az_byJEAusNtcJD1p42xFHxcOE4...
3,RANKED_SOLO_5x5,SILVER,III,flkyRoBSynNfGh-xFarzBAWZMex0umfiKlJcW_gfNYhaLTbx,나는미래의대머리,zTF6cY8vhLTwcxC60p38qIgapinuUIF3RKt-p122QbSdTd...
4,RANKED_SOLO_5x5,SILVER,III,zp2zKTAMW1ubsdh0poGqEtu3fdrjf7pFijHykilKgakBBmv3,졸라못한 내동생,vP8DvOICMzKdLJRVyYskmeVEti9pat7Y56wktAC5P6-d1D...
5,RANKED_SOLO_5x5,SILVER,III,fnajZjq4QKEy0r2kaEKJd8AiZT641vHKT1dgTgZJzGMK-IGM,성냥붙이는소녀,tR40Te-I-ZILm5roQvBTt4gpofSzOErkxKGlhTo85BLAXt...
6,RANKED_SOLO_5x5,SILVER,III,DP-ug1F1b5Rv545iKjDrSEFIXriW3PSzRdIVZ3UJod3PEOHq,균발바닥,wvI3ees2tkS40njUHC5KLQyOlXzHJPqGD6AsU55budXGFq...
7,RANKED_SOLO_5x5,SILVER,III,r1tc_HJcVE10yB443RYAEAOel9dKu9ILiA7za_9ExP2sNE4t,엉덩이흔드러박,T3ACsjA_FPomGgPnlveMEg3v_HqV8rSiWJr7xLBMILOGYI...
8,RANKED_SOLO_5x5,SILVER,III,j3e90G1oWljYSfnYm6zbei2_p4Rb76S3SUJ9untzQcDmMOUj,dding99,oANJUULjOLypvbXflRS_iz2cTZQ6G5qQnqXWA7kmiWNi2Z...
9,RANKED_SOLO_5x5,SILVER,III,VtHAabJCkcu0RDsc3TyYX6TjKcOBJuuZIZxJ9sXcc1RoAP1y,스파르타식판테온,dnbhgvOfNvu5DVmxy0au_UPnjv6bnd3OK6Y9KmndWbYtEQ...


### "accountId" to extract "gameId"
* 위에서 수집한 'accountId'를 이용하여 해당 유저가 진행했던 경기의 고유키 값인 "gameId"를 수집할 수 있다. 이제 해당 "gameId"를 통해 silver 랭크의 게임 데이터, 앞선 분석에서 사용했던 match_data를 수집할 수 있다.
* https://developer.riotgames.com/apis#match-v4

In [129]:
match_info_df = pd.DataFrame()
season = str(13)
for i in range(len(silver_3)):
    try:
        match = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + silver_3['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match)
        print(i, "first_request: ",r.status_code)
        while r.status_code == 429:
            print(r.status_code)
            time.sleep(120)
            match = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + silver_3['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match)

        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    
    except:
        print(i)

0 first_request:  429
429
1 first_request:  200
2 first_request:  200
3 first_request:  200
4 first_request:  200
5 first_request:  200
6 first_request:  200
7 first_request:  200
8 first_request:  200
9
10 first_request:  200
11 first_request:  200
12 first_request:  200
13 first_request:  200
14 first_request:  200
15 first_request:  200
16 first_request:  200
17 first_request:  200
18
19 first_request:  200
20
21 first_request:  200
22 first_request:  200
23 first_request:  200
24 first_request:  200
25 first_request:  200
26 first_request:  200
27 first_request:  200
28 first_request:  200
29 first_request:  200
30 first_request:  200
31 first_request:  200
32 first_request:  200
33 first_request:  200
34 first_request:  200
35 first_request:  200
36 first_request:  200
37 first_request:  200
38 first_request:  200
39 first_request:  200
40 first_request:  200
41 first_request:  200
42 first_request:  200
43 first_request:  200
44 first_request:  200
45 first_request:  200
46 first

In [130]:
match_info_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,5141613485,81,420,13,1618848699870,DUO_CARRY,BOTTOM
1,KR,5141529218,81,420,13,1618846515899,DUO_CARRY,BOTTOM
2,KR,5105040022,81,420,13,1617294278656,DUO_CARRY,BOTTOM
3,KR,5104714480,81,420,13,1617291295151,DUO_CARRY,BOTTOM
4,KR,5104548583,18,420,13,1617289056358,DUO_CARRY,BOTTOM
...,...,...,...,...,...,...,...,...
95,KR,5164999772,45,450,13,1619852380219,DUO_SUPPORT,TOP
96,KR,5164944898,122,450,13,1619850938488,DUO_SUPPORT,MID
97,KR,5164940436,84,450,13,1619849658353,DUO_SUPPORT,NONE
98,KR,5164925168,81,450,13,1619848403253,DUO_SUPPORT,NONE


#### gameId 중복 확인

In [132]:
match_info_df2 = match_info_df.copy()
match_info_df2.duplicated("gameId").sum()

0

### LOL developer port Response Codes(error codes)
* LOL 개발자 포털에서는 100개의 row를 수집하면 2분 동안 request를 막기 때문에 time의 sleep 함수를 이용 (code 429)
* code 429 이외의 error가 발생하는 것에 대비해 시간차를 둔다
* https://developer.riotgames.com/docs/portal#product-registration_application-process

In [133]:
match_fin = pd.DataFrame()
for i in range(len(match_info_df2)):    
    
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_info_df2['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass

    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        print('loop: ',i)
        start_time = time.time()

        while True: # 429error 2분간 대기
            if r.status_code == 429:

                print('request after 2 minutes')
                time.sleep(120)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: #다시 response 200이면 loop escape
                print('total wait time: ', time.time() - start_time)
                print('recovery api cost')
                break

    elif r.status_code == 503: # 잠시 서비스를 이용하지 못하는 에러
        print('service available error')
        start_time = time.time()

        while True:
            if r.status_code == 503 or r.status_code == 429:

                print('request after 2 minutes')
                time.sleep(120)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: 
                print('total error wait time: ', time.time() - start_time)
                print('recovery api cost')
                break
    elif r.status_code == 403: # api갱신이 필요
        print('require new api')
        print('break')
        break

    # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성
    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match_fin = pd.concat([match_fin,mat])


api cost full : infinite loop start
loop location :  4
try 120 second wait time
200
total wait time :  120.33660364151001
recovery api cost
api cost full : infinite loop start
loop location :  104
try 120 second wait time
200
total wait time :  120.35713386535645
recovery api cost
api cost full : infinite loop start
loop location :  204
try 120 second wait time
200
total wait time :  122.36825561523438
recovery api cost
api cost full : infinite loop start
loop location :  304
try 120 second wait time
200
total wait time :  120.37134790420532
recovery api cost
api cost full : infinite loop start
loop location :  404
try 120 second wait time
200
total wait time :  120.33673691749573
recovery api cost
api cost full : infinite loop start
loop location :  504
try 120 second wait time
200
total wait time :  120.36341190338135
recovery api cost
api cost full : infinite loop start
loop location :  604
try 120 second wait time
200
total wait time :  120.36646175384521
recovery api cost
api cost

api cost full : infinite loop start
loop location :  5602
try 120 second wait time
200
total wait time :  120.33053684234619
recovery api cost
api cost full : infinite loop start
loop location :  5702
try 120 second wait time
200
total wait time :  120.38538527488708
recovery api cost
api cost full : infinite loop start
loop location :  5802
try 120 second wait time
200
total wait time :  120.39966821670532
recovery api cost
api cost full : infinite loop start
loop location :  5902
try 120 second wait time
200
total wait time :  120.35340023040771
recovery api cost
api cost full : infinite loop start
loop location :  6002
try 120 second wait time
200
total wait time :  120.36377787590027
recovery api cost
api cost full : infinite loop start
loop location :  6102
try 120 second wait time
200
total wait time :  120.35084676742554
recovery api cost
api cost full : infinite loop start
loop location :  6202
try 120 second wait time
200
total wait time :  120.3932375907898
recovery api cost


#### 랭크 게임 추출
* LOL에는 다양한 게임 방식이 있음 (Rank game, URF, 일반, 칼바람 등등)
* 수집한 "gameId"로 부터 rank 게임만 추출 
    * 랭크 게임은 경기 시작시 챔피언을 선택할 수 없게 만드는 ban 시스템이 존재, 따라서 "ban"이 비어있는 column은 제외

In [ ]:
def get_rank_games_idx(df):
    df_tmp = df.copy()
    tmp = pd.DataFrame(dict(df_tmp["teams"])).T
    idx = []
    
    for i in range(len(tmp)):
        if tmp[0][i].get("bans") == []:
            idx.append(i)
    return idx

idx = get_rank_games_idx(match_fin)

In [ ]:
match_fin = match_fin.loc[match_fin["gameMode"] == "CLASSIC"]
match_fin = match_fin.drop(idx, axis = 0).reset_index(drop = True)
match_fin = match_fin.loc[match_fin["gameDuration"]>=400]
match_fin = match_fin.sort_values("gameId").reset_index(drop = True)
# match_fin.to_csv('silver_data.csv',index=False,encoding = 'cp949')

# Silver Rank Data 전처리

* 이제 수집한 silver data (match_fin)은 기존 master, challenger rank 데이터 분석에 사용했던 raw data와 동일한 형태
* 따라서 기존에 사용했던 전처리 코드 이용

In [434]:
match_fin

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,4488596337,KR,1593874971222,2426,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,4488825341,KR,1593877666346,2152,420,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,4521586060,KR,1595336359344,1685,440,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,4521732452,KR,1595338402306,2323,440,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4521924468,KR,1595341583562,1193,440,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,5356451068,KR,1627536085822,1835,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5573,5356512753,KR,1627540904806,1344,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5574,5356541615,KR,1627540589090,1917,420,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5575,5356582869,KR,1627541133755,1876,440,11,13,11.15.389.2308,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [427]:
win_lose_all = pd.DataFrame(dict(match_fin['teams'])).T 
win_lose_all

,0,1
0,"{'teamId': 100, 'win': 'Fail', 'firstBlood': T...","{'teamId': 200, 'win': 'Win', 'firstBlood': Fa..."
1,"{'teamId': 100, 'win': 'Win', 'firstBlood': Fa...","{'teamId': 200, 'win': 'Fail', 'firstBlood': T..."
2,"{'teamId': 100, 'win': 'Fail', 'firstBlood': T...","{'teamId': 200, 'win': 'Win', 'firstBlood': Fa..."
3,"{'teamId': 100, 'win': 'Fail', 'firstBlood': T...","{'teamId': 200, 'win': 'Win', 'firstBlood': Fa..."
4,"{'teamId': 100, 'win': 'Fail', 'firstBlood': F...","{'teamId': 200, 'win': 'Win', 'firstBlood': Tr..."
...,...,...
5572,"{'teamId': 100, 'win': 'Win', 'firstBlood': Tr...","{'teamId': 200, 'win': 'Fail', 'firstBlood': F..."
5573,"{'teamId': 100, 'win': 'Fail', 'firstBlood': F...","{'teamId': 200, 'win': 'Win', 'firstBlood': Tr..."
5574,"{'teamId': 100, 'win': 'Win', 'firstBlood': Fa...","{'teamId': 200, 'win': 'Fail', 'firstBlood': T..."
5575,"{'teamId': 100, 'win': 'Fail', 'firstBlood': F...","{'teamId': 200, 'win': 'Win', 'firstBlood': Tr..."


In [428]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
for i in range(len(win_lose_all)):
    df1 = df1.append(pd.DataFrame(win_lose_all[0][i]).loc[[0],:])
    df2 = df2.append(pd.DataFrame(win_lose_all[1][i]).loc[[0],:])
df1 = df1.reset_index(drop = True)
df2 = df2.reset_index(drop = True)
df1 = pd.concat([df1, match_fin[["gameId"]]], axis = 1)
df2 = pd.concat([df2, match_fin[["gameId"]]], axis = 1)
df_all = pd.concat([df1, df2])
winner = df_all.loc[df_all["win"] == "Win"].reset_index(drop = True)
loser = df_all.loc[df_all["win"] == "Fail"].reset_index(drop = True)
winner = winner.sort_values(by = "gameId").reset_index(drop = True)
loser = loser.sort_values(by = "gameId").reset_index(drop = True)

winner.to_csv("silver_winner.csv",index=False,encoding = 'cp949')
loser.to_csv("silver_loser.csv",index=False,encoding = 'cp949')

In [436]:
winner

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId
0,100,Win,True,False,False,False,False,False,9,2,1,5,0,0,0,"{'championId': 60, 'pickTurn': 1}",4488596337
1,200,Win,True,True,False,False,True,True,11,3,0,2,0,2,0,"{'championId': 4, 'pickTurn': 6}",4488825341
2,100,Win,False,True,True,True,True,False,8,1,1,3,0,1,0,"{'championId': 105, 'pickTurn': 1}",4521586060
3,200,Win,False,False,False,True,False,False,9,2,2,3,0,0,0,"{'championId': 53, 'pickTurn': 6}",4521732452
4,100,Win,True,True,False,False,True,True,4,0,0,3,0,2,0,"{'championId': 105, 'pickTurn': 1}",4521924468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,200,Win,False,False,False,False,False,False,3,0,0,4,0,0,0,"{'championId': 3, 'pickTurn': 6}",5356451068
5573,200,Win,True,True,False,True,True,False,4,0,1,3,0,0,0,"{'championId': 91, 'pickTurn': 6}",5356512753
5574,200,Win,True,True,True,True,True,True,7,1,1,3,0,1,0,"{'championId': 89, 'pickTurn': 6}",5356541615
5575,200,Win,True,True,True,False,True,True,9,3,0,4,0,2,0,"{'championId': 238, 'pickTurn': 6}",5356582869


---

In [440]:
stat_cols_convert = ['participantId', 'win',
        "kill",
        "vision",
        "timeCC", "tTimeCC",
        "goldE", "goldS",
        "minionK", "nMinionK",
        "champLv",
        "wardsP", "wardsK"
        ]

In [441]:
## stat dict 에서 사용할 컬럼
stat_cols = ['participantId', 'win',
        "kills",
        "visionScore",
        "timeCCingOthers", "totalTimeCrowdControlDealt",
        "goldEarned", "goldSpent",
        "totalMinionsKilled", "neutralMinionsKilled",
        "champLevel",
        "wardsPlaced", "wardsKilled"
        ]

In [439]:
match_data = match_fin.copy()
winner = winner.copy()
loser = loser.copy()
winner.drop(["dominionVictoryScore", "bans"], axis = 1, inplace = True)
loser.drop(["dominionVictoryScore", "bans"], axis = 1, inplace = True)

In [442]:
wl_all = pd.concat([winner, loser])
win = wl_all.loc[wl_all["teamId"] == 100]
lose = wl_all.loc[wl_all["teamId"] == 200]

win = win.set_index("gameId").add_prefix("100_")
lose = lose.set_index("gameId").add_prefix("200_")

winner = pd.concat([win,lose], axis = 1).reset_index()
winner = winner.drop(["100_teamId","100_win","200_teamId","200_win","100_vilemawKills","200_vilemawKills" ], axis = 1)
winner

,gameId,100_firstBlood,100_firstTower,100_firstInhibitor,100_firstBaron,100_firstDragon,100_firstRiftHerald,100_towerKills,100_inhibitorKills,100_baronKills,...,200_firstTower,200_firstInhibitor,200_firstBaron,200_firstDragon,200_firstRiftHerald,200_towerKills,200_inhibitorKills,200_baronKills,200_dragonKills,200_riftHeraldKills
0,4488596337,True,False,False,False,False,False,9,2,1,...,True,True,True,True,True,7,2,1,1,2
1,4488825341,False,False,True,False,False,False,4,1,0,...,True,False,False,True,True,11,3,0,2,2
2,4521586060,False,True,True,True,True,False,8,1,1,...,False,False,False,False,True,0,0,0,1,1
3,4521732452,True,True,True,False,True,True,7,1,0,...,False,False,True,False,False,9,2,2,3,0
4,4521924468,True,True,False,False,True,True,4,0,0,...,False,False,False,False,False,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,5356451068,True,True,False,False,True,True,5,0,0,...,False,False,False,False,False,3,0,0,4,0
5573,5356512753,False,False,False,False,False,False,0,0,0,...,True,False,True,True,False,4,0,1,3,0
5574,5356541615,False,False,False,False,False,False,1,0,0,...,True,True,True,True,True,7,1,1,3,1
5575,5356582869,False,False,False,False,False,False,2,0,0,...,True,True,False,True,True,9,3,0,4,2


In [443]:
match_data.drop(["gameMode","gameVersion", "mapId","queueId","seasonId","gameType" ], axis = 1, inplace = True)


,gameId,platformId,gameCreation,gameDuration,teams,participants,participantIdentities
0,4488596337,KR,1593874971222,2426,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,4488825341,KR,1593877666346,2152,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,4521586060,KR,1595336359344,1685,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,4521732452,KR,1595338402306,2323,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4521924468,KR,1595341583562,1193,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...,...,...,...
5572,5356451068,KR,1627536085822,1835,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5573,5356512753,KR,1627540904806,1344,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5574,5356541615,KR,1627540589090,1917,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
5575,5356582869,KR,1627541133755,1876,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [444]:
match_data = match_data.merge(winner, how = "left", on = "gameId")
match_data

,gameId,platformId,gameCreation,gameDuration,teams,participants,participantIdentities,100_firstBlood,100_firstTower,100_firstInhibitor,...,200_firstTower,200_firstInhibitor,200_firstBaron,200_firstDragon,200_firstRiftHerald,200_towerKills,200_inhibitorKills,200_baronKills,200_dragonKills,200_riftHeraldKills
0,4488596337,KR,1593874971222,2426,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",True,False,False,...,True,True,True,True,True,7,2,1,1,2
1,4488825341,KR,1593877666346,2152,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",False,False,True,...,True,False,False,True,True,11,3,0,2,2
2,4521586060,KR,1595336359344,1685,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",False,True,True,...,False,False,False,False,True,0,0,0,1,1
3,4521732452,KR,1595338402306,2323,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",True,True,True,...,False,False,True,False,False,9,2,2,3,0
4,4521924468,KR,1595341583562,1193,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",True,True,False,...,False,False,False,False,False,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,5356451068,KR,1627536085822,1835,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",True,True,False,...,False,False,False,False,False,3,0,0,4,0
5573,5356512753,KR,1627540904806,1344,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",False,False,False,...,True,False,True,True,False,4,0,1,3,0
5574,5356541615,KR,1627540589090,1917,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",False,False,False,...,True,True,True,True,True,7,1,1,3,1
5575,5356582869,KR,1627541133755,1876,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':...",False,False,False,...,True,True,False,True,True,9,3,0,4,2


In [449]:
# for i in len(match_df_v1):
#     pd.DataFrame(match_df_v1.participants[i])["stats"]

df = pd.DataFrame()
for i in range(len(match_data)):
    tmp = pd.DataFrame(match_data.participants[i])
    df_stats = tmp["stats"].apply(lambda x: pd.Series(x))
    df_stats = df_stats.loc[:,stat_cols]
    df_stats.columns = stat_cols_convert
    tmp = tmp.merge(df_stats,how = "inner", on ="participantId" )
    tmp = tmp.drop(["spell1Id", "spell2Id","stats","timeline"], axis = 1)
    win_lose = tmp.groupby("teamId").agg(win = ('win' , 'mean'),
                                         kill = ('kill' , 'sum'),
                                         vision = ('vision', 'sum'),
                                         timeCC = ('timeCC', 'sum'),
                                         tTimeCC =  ('tTimeCC', 'sum'),
                                         goldE =('goldE', 'sum'),
                                         goldE_std = ('goldE', 'std'),
                                         goldS = ('goldS', 'sum'),
                                         minionK = ('minionK', 'sum'),
                                         nMinionK = ('nMinionK', 'sum'),
                                         champLv = ('champLv', 'mean'),
                                         champLv_std =  ('champLv', 'std'),
                                         wardsP =  ('wardsP', 'sum'),
                                         wardsK =   ('wardsK', 'sum')
                                        ).reset_index()

    team_100 = win_lose[win_lose["teamId"] == 100].add_prefix("100_")
    team_200 = win_lose[win_lose["teamId"] == 200].add_prefix("200_").reset_index(drop = True)
    team_stat = pd.concat([team_100, team_200], axis = 1)
    df = df.append(team_stat)

df = df.reset_index(drop = True)

In [453]:
# df_match = pd.concat([match_data, df.reset_index(drop = True)], axis = 1)
df_match = pd.concat([match_data,df], axis = 1)


In [454]:
df_match = df_match.drop(["teams","gameCreation","platformId","200_win",
                          "100_teamId","200_teamId","participantIdentities","participants"
                          ], axis = 1)

In [463]:
pd.options.display.max_columns = None
print(df_match.isna().sum().sum())
df_match

0


,gameId,gameDuration,100_firstBlood,100_firstTower,100_firstInhibitor,100_firstBaron,100_firstDragon,100_firstRiftHerald,100_towerKills,100_inhibitorKills,100_baronKills,100_dragonKills,100_riftHeraldKills,200_firstBlood,200_firstTower,200_firstInhibitor,200_firstBaron,200_firstDragon,200_firstRiftHerald,200_towerKills,200_inhibitorKills,200_baronKills,200_dragonKills,200_riftHeraldKills,100_win,100_kill,100_vision,100_timeCC,100_tTimeCC,100_goldE,100_goldE_std,100_goldS,100_minionK,100_nMinionK,100_champLv,100_champLv_std,100_wardsP,100_wardsK,200_kill,200_vision,200_timeCC,200_tTimeCC,200_goldE,200_goldE_std,200_goldS,200_minionK,200_nMinionK,200_champLv,200_champLv_std,200_wardsP,200_wardsK
0,4488596337,2426,True,False,False,False,False,False,9,2,1,5,0,False,True,True,True,True,True,7,2,1,1,2,True,31,213,57,1088,73537,2422.813612,69583,823,261,17.6,0.894427,78,34,21,207,99,911,67357,3732.031859,63898,776,177,16.6,1.140175,104,26
1,4488825341,2152,False,False,True,False,False,False,4,1,0,3,0,True,True,False,False,True,True,11,3,0,2,2,False,27,238,142,2123,59491,2817.598747,56810,676,170,15.2,1.483240,99,34,33,201,82,662,67161,3397.003636,60176,706,208,16.0,1.581139,81,30
2,4521586060,1685,False,True,True,True,True,False,8,1,1,3,1,True,False,False,False,False,True,0,0,0,1,1,True,29,175,110,1017,56547,1706.534734,50833,657,182,14.8,1.303840,67,40,14,143,56,633,45703,2149.951232,44695,619,134,13.4,1.140175,77,24
3,4521732452,2323,True,True,True,False,True,True,7,1,0,3,2,False,False,False,True,False,False,9,2,2,3,0,False,26,263,189,1486,65997,2359.226738,63638,801,201,15.6,1.949359,114,44,33,228,202,2318,70884,3014.129509,61875,703,196,17.0,1.224745,86,42
4,4521924468,1193,True,True,False,False,True,True,4,0,0,3,2,False,False,False,False,False,False,0,0,0,0,0,True,21,81,84,688,39277,1895.829581,32643,461,113,11.2,1.923538,39,10,9,70,47,774,28499,722.960718,26310,352,93,10.0,1.000000,34,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,5356451068,1835,True,True,False,False,True,True,5,0,0,1,2,False,False,False,False,False,False,3,0,0,4,0,False,23,157,94,822,53306,2314.654877,51940,612,216,13.6,1.140175,55,20,35,129,76,1060,53619,1922.041805,50650,432,167,14.2,1.303840,61,14
5573,5356512753,1344,False,False,False,False,False,False,0,0,0,0,0,True,True,False,True,True,False,4,0,1,3,0,False,9,78,56,277,31076,1123.505096,28910,411,95,11.2,1.788854,33,9,16,70,83,896,40478,1947.714892,34510,498,147,12.4,1.140175,38,4
5574,5356541615,1917,False,False,False,False,False,False,1,0,0,1,1,True,True,True,True,True,True,7,1,1,3,1,False,23,144,118,1104,52688,1908.947563,50670,639,152,14.4,1.140175,65,24,32,170,161,901,59316,2534.707518,54705,648,186,15.6,1.341641,60,31
5575,5356582869,1876,False,False,False,False,False,False,2,0,0,0,0,True,True,True,False,True,True,9,3,0,4,2,False,27,98,129,891,50426,2057.019008,47925,518,73,13.6,0.894427,55,7,41,73,83,417,59113,4320.065312,52830,490,190,14.6,1.673320,40,5


In [464]:
df_match.to_csv("silver_df_match.csv", index = False, encoding = "cp949")

In [8]:
silver_loser = pd.read_csv("./data/silver_loser.csv")
silver_winner = pd.read_csv("./data/silver_winner.csv")
df_match_silver = pd.read_csv("./data/silver_df_match.csv")

## silver rank match data 최종 형태

In [9]:
df_match_silver

,gameId,gameDuration,100_firstBlood,100_firstTower,100_firstInhibitor,100_firstBaron,100_firstDragon,100_firstRiftHerald,100_towerKills,100_inhibitorKills,...,200_tTimeCC,200_goldE,200_goldE_std,200_goldS,200_minionK,200_nMinionK,200_champLv,200_champLv_std,200_wardsP,200_wardsK
0,4488596337,2426,True,False,False,False,False,False,9,2,...,911,67357,3732.031859,63898,776,177,16.6,1.140175,104,26
1,4488825341,2152,False,False,True,False,False,False,4,1,...,662,67161,3397.003636,60176,706,208,16.0,1.581139,81,30
2,4521586060,1685,False,True,True,True,True,False,8,1,...,633,45703,2149.951232,44695,619,134,13.4,1.140175,77,24
3,4521732452,2323,True,True,True,False,True,True,7,1,...,2318,70884,3014.129509,61875,703,196,17.0,1.224745,86,42
4,4521924468,1193,True,True,False,False,True,True,4,0,...,774,28499,722.960718,26310,352,93,10.0,1.000000,34,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5572,5356451068,1835,True,True,False,False,True,True,5,0,...,1060,53619,1922.041805,50650,432,167,14.2,1.303840,61,14
5573,5356512753,1344,False,False,False,False,False,False,0,0,...,896,40478,1947.714892,34510,498,147,12.4,1.140175,38,4
5574,5356541615,1917,False,False,False,False,False,False,1,0,...,901,59316,2534.707518,54705,648,186,15.6,1.341641,60,31
5575,5356582869,1876,False,False,False,False,False,False,2,0,...,417,59113,4320.065312,52830,490,190,14.6,1.673320,40,5
